# 01. Segmentation

## 01-2. Segmentation 개념 및 적용
### Segmentation이란?
Segmentation(분할)은 이미지 내의 개별 객체나 영역을 구분하는 과정입니다. 대표적으로 다음과 같은 방식이 있습니다.
- **Semantic Segmentation**: 같은 클래스의 픽셀을 그룹화 (예: 하늘, 도로, 사람 등)
- **Instance Segmentation**: 개별 객체를 분리 (예: 각각의 사람, 자동차 구분)

### YOLO-Seg vs. 기존 Segmentation 기법
| 기법 | 설명 | 장점 | 단점 |
|------|------|------|------|
| U-Net | CNN 기반 의료영상, 일반 이미지 분할에 활용 | 높은 정확도, 경량화 가능 | 비교적 속도가 느림 |
| DeepLabV3+ | Atrous convolution 활용, 고해상도 이미지 분할 가능 | 높은 정확도 | 모델 크기가 크고 연산량이 많음 |
| YOLO-Seg | YOLO 기반 실시간 객체 분할 | 빠른 속도, 객체 감지와 분할 동시 수행 | 기존 Segmentation보다 세밀한 영역 구분이 약할 수 있음 |

### 라즈베리파이에서 사용할 수 있는 Segmentation 모델
- **TensorFlow Lite(TFLite) 기반 모델**: MobileNet+DeepLabV3
- **YOLO-Seg TFLite 변환 후 사용**

# 02. [실습 8] TFLite 기반 YOLO-Seg 실행
1. 라즈베리파이에서 YOLO-Seg 모델 다운로드 및 변환
2. OpenCV 및 TFLite로 Segmentation 실행
3. 결과 시각화 및 성능 측정

## 02-1. YOLOv5n을 활용한 실시간 Segmentation

### YOLOv5 git 다운로드 및 라이브러리 설치
```bash
# YOLOv5 저장소 클론
git clone https://github.com/ultralytics/yolov5.git
cd yolov5

# 의존성 패키지 설치 (라즈베리파이에 필요한 라이브러리 포함)
pip install -r requirements.txt
```

### yolov5n-seg 모델 다운로드
```bash
mkdir weights
wget -O weights/yolov5n-seg.pt https://github.com/ultralytics/yolov5/releases/download/v6.2/yolov5n-seg.pt
```

### YOLOv5s 모델을 tflite 모델로 변환
```bash
python3 export.py --weights weights/yolov5n-seg.pt \
                 --include tflite \
                 --img 640 \
                 --device cpu

```

### YOLO-seg를 실행하기 위한 라이브러리 설치
```bash
pip install numpy opencv-python tflite-runtime
```

### YOLO-seg 실행

In [3]:
import cv2
import numpy as np
import tensorflow.lite as tflite

# YOLOv5n-seg TFLite 모델 로드
interpreter = tflite.Interpreter(model_path="./yolov5/yolov5n-seg-fp16.tflite")
interpreter.allocate_tensors()

# 입력 및 출력 텐서 정보 가져오기
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# 정확도를 높이기 위해 Threshold 조정
THRESHOLD = 0.5  

# 투명도 조절 (0.0 ~ 1.0)
ALPHA = 0.5  

# YOLO 모델 해상도 설정 (원본 비율 유지 X, YOLO 학습 크기 유지)
YOLO_SIZE = 640  

# 이미지 전처리 함수 (YOLO 모델이 학습한 방식과 동일하게 적용)
def preprocess_image(image_path, input_shape):
    img = cv2.imread(image_path)

    # YOLO 입력 크기(640x640)로 강제 리사이즈 (원본 비율 유지 X)
    img_resized = cv2.resize(img, (YOLO_SIZE, YOLO_SIZE), interpolation=cv2.INTER_AREA)

    # YOLO 모델 입력 형식 변환
    img_yolo = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
    img_yolo = np.expand_dims(img_yolo, axis=0)  

    return img_yolo, img_resized  # YOLO 입력용 이미지와 원본 리사이즈된 이미지 반환

# 세그멘테이션 실행 함수
def run_inference(image_path):
    input_shape = input_details[0]['shape']
    
    # 이미지 전처리 (YOLO 입력 크기 유지)
    input_data, resized_image = preprocess_image(image_path, input_shape)
    
    # 모델에 입력 데이터 설정
    interpreter.set_tensor(input_details[0]['index'], input_data)
    
    # 모델 실행
    interpreter.invoke()
    
    # 결과 가져오기
    segmentation_output = interpreter.get_tensor(output_details[0]['index'])

    # 첫 번째 클래스 선택
    segmentation_mask = segmentation_output[0, 0, :, :]  

    # Threshold 적용
    segmentation_mask = (segmentation_mask > THRESHOLD).astype(np.uint8) * 255  

    # Gaussian Blur 적용
    segmentation_mask = cv2.GaussianBlur(segmentation_mask, (5, 5), 0)

    # 원본 크기에 맞춰 리사이즈 (원본 비율이 아닌 YOLO 해상도로 유지)
    mask_overlay = cv2.resize(segmentation_mask, (YOLO_SIZE, YOLO_SIZE), interpolation=cv2.INTER_NEAREST)

    # 컬러 마스크 생성
    mask_colored = np.zeros((YOLO_SIZE, YOLO_SIZE, 3), dtype=np.uint8)
    mask_colored[mask_overlay > 128] = [0, 255, 0]  

    # 원본 이미지와 투명한 마스크 합성
    overlay_image = cv2.addWeighted(resized_image, 1 - ALPHA, mask_colored, ALPHA, 0)

    return overlay_image

# 실행 테스트
image_path = "test.jpg"
overlay_result = run_inference(image_path)

# 결과 시각화
cv2.imshow("Segmentation Overlay", overlay_result)
cv2.waitKey(0)
cv2.destroyAllWindows()


2025-02-17 15:19:05.721703: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739773145.731160   24149 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739773145.733779   24149 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# 03. Tracking
## 03-1. Tracking 개념 및 적용
### Object Tracking이란?
Tracking(추적)은 비디오 프레임에서 객체를 식별하고 연속된 프레임에서 해당 객체의 위치를 추적하는 기술입니다. 컴퓨터 비전에서는 다양한 방식의 트래킹 기법이 존재하며, 각각의 알고리즘은 상황에 따라 적절하게 사용됩니다.
- **Single Object Tracking(SOT)**: 한 개의 객체를 추적 (e.g., KCF, CSRT)
- **Multi-Object Tracking(MOT)**: 여러 객체를 동시에 추적 (e.g., SORT, DeepSORT, ByteTrack)

## 03-2. Tracking의 기본 원리
트래킹은 기본적으로 객체 탐지(Object Detection) + 연속된 프레임에서의 위치 예측으로 이루어집니다.

### 객체 탐지 (Object Detection)
객체를 추적하려면 먼저 객체를 탐지해야 합니다. 일반적으로 YOLO, Faster R-CNN, SSD 같은 딥러닝 기반 모델이 사용됩니다.
- **한 번만 탐지(One-shot Detection)**: 첫 프레임에서만 객체를 탐지하고 이후에는 위치를 예측하여 추적 (e.g., Kalman Filter)
- **매 프레임 탐지(Frame-by-Frame Detection)**: 매 프레임마다 객체 탐지를 수행하여 위치를 갱신 (e.g., YOLO + Tracking)

### 객체 매칭 (Object Association)
이전 프레임과 현재 프레임의 객체를 연결하는 과정입니다. 트래킹에서는 다음과 같은 방식이 사용됩니다.
- **IOU (Intersection over Union)**: 두 개의 바운딩 박스의 겹치는 영역을 비교하여 같은 객체로 인식
- **Feature Matching**: 객체의 특징점(ORB, SIFT, SURF)을 비교하여 동일한 객체 판별
- **딥러닝 기반 Re-identification (Re-ID)**: CNN을 이용하여 객체를 구별하고 동일 객체인지 판별

### 예측 및 보정
객체의 위치는 다음 프레임에서 어디에 있을지를 예측하고, 이후 실제 측정값을 기반으로 보정됩니다.
- **칼만 필터 (Kalman Filter)**: 선형적으로 객체의 움직임을 예측하고 보정
- **파티클 필터 (Particle Filter)**: 확률 분포를 활용하여 비선형적인 움직임까지 추적 가능
- **LSTM 기반 예측**: 객체의 이동 패턴을 학습하여 예측 정확도를 높임

### 객체 추적의 필요성
객체 추적은 다음과 같은 다양한 분야에서 활용됩니다.
- **자율 주행 차량**: 차량과 보행자의 움직임을 추적하여 안전한 주행을 지원
- **스포츠 분석**: 선수들의 움직임을 추적하여 경기 데이터를 분석
- **감시 시스템**: 특정 인물 또는 차량의 이동을 모니터링
- **로봇 비전**: 물체를 따라가거나 특정한 대상과 상호작용
- **증강 현실 (AR)**: 실시간으로 사용자의 움직임을 추적하여 자연스러운 인터페이스 제공

## 03-3. Tracking 알고리즘
객체 추적을 위한 대표적인 알고리즘에는 여러 가지가 있으며, 용도에 따라 적절한 방식을 선택해야 합니다.

### 전통적인 트래킹 방법
| 알고리즘 | 설명 |
|----------|------|
| **Mean-Shift** | 히스토그램을 기반으로 이동하는 객체의 위치를 찾음 |
| **CamShift** | Mean-Shift를 개선하여 크기 변화까지 추적 가능 |
| **Optical Flow** | 프레임 간 픽셀 이동을 기반으로 객체의 움직임을 추적 |
| **Kalman Filter** | 선형 움직임을 가정하고 객체의 다음 위치를 예측 |
| **Particle Filter** | 확률 모델을 활용하여 비선형 움직임을 추적 |

### 딥러닝 기반 트래킹 방법
| 알고리즘 | 설명 |
|----------|------|
| **SORT (Simple Online and Realtime Tracker)** | Kalman Filter + IOU 기반 추적 |
| **DeepSORT** | SORT에 Re-ID를 추가하여 재인식 성능 향상 |
| **ByteTrack** | SORT의 탐지를 개선하여 신뢰도 낮은 객체도 추적 |
| **FairMOT** | Multi-Object Tracking을 위한 단일 네트워크 적용 |
| **Siamese Networks** | 쌍둥이 네트워크를 사용하여 객체를 계속 인식 |
| **Transformer 기반 트래킹 (TrackFormer)** | Transformer 모델을 활용한 최신 트래킹 방식 |

---

# 04. [실습 9] 라즈베리파이에서 Multi-Object Tracking 적용
1. YOLO + SORT를 적용한 MOT 구현
2. OpenCV 기반 실시간 Tracking 수행
3. 성능 비교 및 최적화 방안 분석

### 필수 라이브러리 설치
```bash
sudo apt update && sudo apt upgrade -y
sudo apt install python3-pip python3-opencv libatlas-base-dev -y
pip3 install torch torchvision numpy opencv-python
```

### YOLOv5 설치
```bash
git clone https://github.com/ultralytics/yolov5.git
cd yolov5
pip install -r requirements.txt
```

In [26]:
import sys
import torch
import cv2
import numpy as np

# yolov5 폴더를 경로에 추가
sys.path.append("yolov5")

from yolov5.models.common import DetectMultiBackend
from yolov5.utils.general import non_max_suppression

# 라즈베리파이 CPU 모드
device = torch.device("cpu")  # 문자열 "cpu"가 아니라 torch.device 객체 사용
model = DetectMultiBackend("yolov5n.pt", device=device)
model.eval()

print("YOLOv5 모델 로드 완료!")


100%|██████████| 3.87M/3.87M [00:01<00:00, 3.66MB/s]

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs


YOLOv5 모델 로드 완료!


### Tracking 라이브러리 설치

```bash
pip install supervision
```

In [2]:
import sys
import torch
import cv2
import numpy as np
import supervision as sv

sys.path.append("yolov5")
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.general import non_max_suppression
from yolov5.utils.torch_utils import select_device

# YOLOv5 모델 로드
device = select_device('cpu')  # 라즈베리파이에서는 CPU 사용
model = DetectMultiBackend("yolov5n.pt", device=device)
model.eval()

# ByteTrack 트래커 초기화
tracker = sv.ByteTrack()

# ✅ 객체의 궤적(trajectory)을 저장할 딕셔너리
track_history = {}

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv5 입력 전처리
    img = cv2.resize(frame, (640, 640))
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR → RGB 변환
    img = np.ascontiguousarray(img, dtype=np.float32) / 255.0  # 정규화
    img = torch.from_numpy(img).unsqueeze(0).to(device)

    # 객체 탐지 수행
    with torch.no_grad():
        preds = model(img)
        preds = non_max_suppression(preds, conf_thres=0.4, iou_thres=0.5)[0]

    # 트래킹을 위한 Detections 객체 생성
    if preds is not None and len(preds) > 0:
        xyxy = []
        confidences = []
        class_ids = []

        for det in preds:
            x1, y1, x2, y2, conf, cls = det[:6]
            xyxy.append([x1, y1, x2, y2])
            confidences.append(conf.item())  # Tensor → float 변환
            class_ids.append(int(cls))

        detections = sv.Detections(
            xyxy=np.array(xyxy),
            confidence=np.array(confidences),
            class_id=np.array(class_ids),
        )

        # 트래킹 업데이트
        tracks = tracker.update_with_detections(detections)

        # 트래킹된 객체에 ID와 bounding box 표시
        for i in range(len(tracks.xyxy)):
            x1, y1, x2, y2 = map(int, tracks.xyxy[i])  # 바운딩 박스 좌표
            track_id = tracks.tracker_id[i]  # 트래킹 ID
            
            # ✅ ID별로 궤적(trajectory) 저장
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2

            if track_id not in track_history:
                track_history[track_id] = []

            track_history[track_id].append((center_x, center_y))

            # ✅ 궤적(trajectory) 선 그리기
            for j in range(1, len(track_history[track_id])):
                if track_history[track_id][j - 1] is None or track_history[track_id][j] is None:
                    continue
                cv2.line(frame, track_history[track_id][j - 1], track_history[track_id][j], (0, 255, 255), 2)

            # 바운딩 박스 그리기
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 화면 출력
    cv2.imshow("YOLOv5 + Supervision Tracking + Trajectory", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


YOLOv5 🚀 v7.0-398-g5cdad892 Python-3.10.12 torch-2.6.0+cu124 CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs
